<a href="https://colab.research.google.com/github/YugyeongJo/study_data_analytics/blob/main/docs/quests/MLs/RentalCarOfContractType_LinearRegresstion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import pandas as pd

In [51]:
df_RCOCT = pd.read_csv('/content/RentalCarOfContractType.csv')
df_RCOCT[:5]

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,0,없음,5.0,농협은행,정상,60.0,2449.0
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,12,있음,8.0,농협은행,해약,51.0,1942.0


In [52]:
df_RCOCT.shape

(51304, 17)

## features, target 선택
- target : age
- features : Term, amount, overdue_count


## 데이터 전처리(Pre-Processing)

In [53]:
# column 추출
df_RCOCT_extract = df_RCOCT.loc[:, ['age', 'Term', 'amount', 'overdue_count']]
df_RCOCT_extract[:5]

,age,Term,amount,overdue_count
0,43.0,60,96900,0
1,62.0,60,102900,0
2,60.0,60,96900,0
3,60.0,12,66900,0
4,51.0,12,66900,12


In [54]:
# 결측치 확인
df_RCOCT_extract.isnull().sum()
df_RCOCT_clean_extract = df_RCOCT_extract.dropna(subset=['age']).copy()

In [55]:
# 공백 여부 확인
df_RCOCT_clean_extract.sort_values(['age', 'Term', 'amount', 'overdue_count'], ascending=[True, True, True, True])

,age,Term,amount,overdue_count
6645,20.0,12,66900,0
11406,20.0,12,66900,0
11448,20.0,12,66900,0
19059,20.0,12,66900,0
22178,20.0,12,66900,0
...,...,...,...,...
34171,101.0,60,96900,0
6114,103.0,60,96900,0
29158,103.0,60,96900,0
15174,104.0,12,66900,0


In [56]:
# 이상치 확인
df_RCOCT_clean_extract['age'].unique()
df_RCOCT_clean_extract['Term'].unique()
df_RCOCT_clean_extract['amount'].unique()
df_RCOCT_clean_extract['overdue_count'].unique()
df_RCOCT_clean_extract.sort_values('overdue_count')  # 연체횟수가 -1일 수 있을까...?

,age,Term,amount,overdue_count
1917,50.0,60,105900,-1
1920,28.0,60,90900,-1
1859,65.0,60,96900,-1
25677,25.0,60,87900,-1
25619,52.0,60,90900,-1
...,...,...,...,...
6347,33.0,39,120900,15
3091,34.0,60,90900,15
344,45.0,60,90900,15
2702,63.0,60,81900,15


### 데이터 분리
- target과 labels(features) 분리
- train과 test set 분리

In [57]:
target = df_RCOCT_clean_extract['age']
features = df_RCOCT_clean_extract[['Term',	'amount',	'overdue_count']]
target.shape, features.shape  # 행 갯수 일치 확인

((40509,), (40509, 3))

In [58]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=2)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((28356, 3), (12153, 3), (28356,), (12153,))

## 모델

### 목표변수에 따른 모델 선택 - 연속형

In [59]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [60]:
# 학습 진행
model.fit(features_train, target_train)

LinearRegression()

## 평가

In [61]:
from sklearn.metrics import r2_score
target_train_predict = model.predict(features_train)
target_train.shape, target_train_predict.shape

((28356,), (28356,))

In [62]:
# train data set에 대한 평가
r2_score(target_train, target_train_predict)

5.2057096017854576e-05

In [63]:
target_test_predict = model.predict(features_test)
target_test.shape, target_test_predict.shape

((12153,), (12153,))

In [64]:
r2_score(target_test, target_test_predict)

-0.00021001332287418073

## 서비스
- 사용자 입력 시 주의 사항 : 학습 때 사용한 포맷을 그대로 유지

In [65]:
# 결측치 있는 age 추출
df_RCOCT_age_missingvalue = df_RCOCT.query('age.isna()')

In [66]:
# model.predict
predict_age = model.predict(df_RCOCT_age_missingvalue[['Term',	'amount',	'overdue_count']])

In [67]:
# 결측치 넣어주기
df_RCOCT.loc[df_RCOCT['age'].isna(), 'age'] = predict_age

In [68]:
df_RCOCT.isnull().sum()

id                       0
type_of_contract         4
type_of_contract2        1
channel                  0
datetime                 0
Term                     0
payment_type             0
product                  1
amount                   0
state                    0
overdue_count            0
overdue                  2
credit rating         8783
bank                  2760
cancellation            25
age                      0
Mileage              10795
dtype: int64

In [71]:
import pickle

In [73]:
with open('RentalCarOfContractType.pkl', 'wb') as pickle_file : pickle.dump(obj=model, file=pickle_file)

In [74]:
df_RCOCT.to_csv('RentalCarOfContractType_ageRegression.csv', index=False)